# Frisco, TX  Real Estate Web Scraping

## Importing Libraries

In [1]:
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
import pymongo

## Setting up browser to scrape data using Splinter

In [2]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/tiffanyelle/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


##  Current Listed Properties

### 1. Extracting Data using Web Scraping

In [3]:
# Visit the following URL
url = "https://www.redfin.com/city/30844/TX/Frisco"
browser.visit(url)

In [4]:
# Using BeautifulSoup to get html.parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Price of listed properties
price_list = []
price = soup.find_all('span', class_="homecardV2Price")
price[:5]

[<span class="homecardV2Price" data-rf-test-name="homecard-price">$564,900</span>,
 <span class="homecardV2Price" data-rf-test-name="homecard-price">$930,000</span>,
 <span class="homecardV2Price" data-rf-test-name="homecard-price">$305,000</span>,
 <span class="homecardV2Price" data-rf-test-name="homecard-price">$549,900</span>,
 <span class="homecardV2Price" data-rf-test-name="homecard-price">$423,550</span>]

In [6]:
# Number of Beds, Baths and Sq. Ft. of listed properties
stats = soup.find_all('div', class_="stats")
stats[:6]

[<div class="stats">3 Beds</div>,
 <div class="stats">2.5 Baths</div>,
 <div class="stats">2,997 Sq. Ft.</div>,
 <div class="stats">3 Beds</div>,
 <div class="stats">3.5 Baths</div>,
 <div class="stats">4,228 Sq. Ft.</div>]

In [7]:
# Zipcode of listed properties
zipcode = soup.find_all('div', class_="link-and-anchor")
zipcode[:5]

[<div class="link-and-anchor">7200 Neches Pine Dr, Frisco, TX 75036</div>,
 <div class="link-and-anchor">4520 Florence Dr, Frisco, TX 75034</div>,
 <div class="link-and-anchor">8016 Cool River Dr, Frisco, TX 75036</div>,
 <div class="link-and-anchor">13942 Badger Creek Dr, Frisco, TX 75033</div>,
 <div class="link-and-anchor">1924 Dexter Ln, Frisco, TX 75036</div>]

### 2. Data Cleaning and Transformation to load into DB

In [8]:
# Convering Price value into numeric - removing "$" and ","
price_list = [float(p.text.replace("$","").replace(",","").replace("+","").strip()) for p in price]
price_list[:5]

[564900.0, 930000.0, 305000.0, 549900.0, 423550.0]

In [9]:
print("Number of records in price list", len(price_list))

Number of records in price list 41


In [10]:
# Convering Number of Beds, Number of Baths and Area Sq. Ft. values into numeric - removing text: "Beds", "Baths", "Sq. Ft."
beds_list = []
baths_list = []
sq_ft_list = []

for each in stats:
    val = each.text.strip()
    if 'Bed' in val:
        beds_list.append(float(val.replace("Beds","").replace("Bed","").strip()))
    elif 'Bath' in val:  
        baths_list.append(float(val.replace("Baths","").replace("Bath","").strip()))
    elif 'Sq. Ft.' in val:
        sq_ft_list.append(float(val.replace("Sq. Ft.","").replace(",","").strip()))

print("Beds List:", beds_list[:5])
print("Baths List:", baths_list[:5])
print("Sq. Ft. List:", sq_ft_list[:5])

Beds List: [3.0, 3.0, 2.0, 5.0, 3.0]
Baths List: [2.5, 3.5, 2.0, 3.5, 2.0]
Sq. Ft. List: [2997.0, 4228.0, 1548.0, 3556.0, 1970.0]


In [11]:
print("Number of records in beds list", len(beds_list))
print("Number of records in baths list", len(baths_list))
print("Number of records in sq. ft. list", len(sq_ft_list))

Number of records in beds list 41
Number of records in baths list 41
Number of records in sq. ft. list 41


In [12]:
#Extracting zipcode from the complete address
zipcode_list = []
zipcode_list =  [z.text[-5:].strip() for z in zipcode]
print("Zipcode List:", zipcode_list[:5])

Zipcode List: ['75036', '75034', '75036', '75033', '75036']


In [13]:
print("Number of records in zipcode list", len(zipcode_list))

Number of records in zipcode list 41


#### Creating a list of Dictionary with current listed real estate properties to load into MongoDB

In [14]:
current_listing = []

for i in range(len(zipcode_list)):
    temp_dict = {"ZipCode":zipcode_list[i], "NumberOfBeds": beds_list[i], "NumberOfBaths": baths_list[i], 
           "SqFt": sq_ft_list[i], "Price":price_list[i]}
    current_listing.append(temp_dict)
    
print("Records to insert in DB:")
print(current_listing[:5])

Records to insert in DB:
[{'ZipCode': '75036', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.5, 'SqFt': 2997.0, 'Price': 564900.0}, {'ZipCode': '75034', 'NumberOfBeds': 3.0, 'NumberOfBaths': 3.5, 'SqFt': 4228.0, 'Price': 930000.0}, {'ZipCode': '75036', 'NumberOfBeds': 2.0, 'NumberOfBaths': 2.0, 'SqFt': 1548.0, 'Price': 305000.0}, {'ZipCode': '75033', 'NumberOfBeds': 5.0, 'NumberOfBaths': 3.5, 'SqFt': 3556.0, 'Price': 549900.0}, {'ZipCode': '75036', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.0, 'SqFt': 1970.0, 'Price': 423550.0}]


### 3. Loading data into Mongo DB

In [15]:
# Making connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'RealEstate' database in Mongo
db = client.RealEstate

In [16]:
# collection 'current_listing'
current_listing_collection = db.current_listing_collection

In [17]:
# Loading Data into collection table
for i in current_listing:
    db.current_listing_collection.insert_one(i)

In [18]:
# checking loaded records
for record in db.current_listing_collection.find():
    print(record)

{'_id': ObjectId('6062623af14bffb3486fd800'), 'ZipCode': '75036', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.5, 'SqFt': 2997.0, 'Price': 564900.0}
{'_id': ObjectId('6062623af14bffb3486fd801'), 'ZipCode': '75034', 'NumberOfBeds': 3.0, 'NumberOfBaths': 3.5, 'SqFt': 4228.0, 'Price': 930000.0}
{'_id': ObjectId('6062623af14bffb3486fd802'), 'ZipCode': '75036', 'NumberOfBeds': 2.0, 'NumberOfBaths': 2.0, 'SqFt': 1548.0, 'Price': 305000.0}
{'_id': ObjectId('6062623af14bffb3486fd803'), 'ZipCode': '75033', 'NumberOfBeds': 5.0, 'NumberOfBaths': 3.5, 'SqFt': 3556.0, 'Price': 549900.0}
{'_id': ObjectId('6062623af14bffb3486fd804'), 'ZipCode': '75036', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.0, 'SqFt': 1970.0, 'Price': 423550.0}
{'_id': ObjectId('6062623af14bffb3486fd805'), 'ZipCode': '75035', 'NumberOfBeds': 4.0, 'NumberOfBaths': 3.0, 'SqFt': 3676.0, 'Price': 599900.0}
{'_id': ObjectId('6062623af14bffb3486fd806'), 'ZipCode': '75036', 'NumberOfBeds': 5.0, 'NumberOfBaths': 3.5, 'SqFt': 3936.0, 'Price': 52

## Sold Properties

### 1. Extracting Data using Web Scraping

In [19]:
# Visit the following URL
url = "https://www.redfin.com/city/30844/TX/Frisco/filter/include=sold-6mo"
browser.visit(url)

In [20]:
# Using BeautifulSoup to get html.parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [21]:
# Price of listed properties
price_list = []
price = soup.find_all('span', class_="homecardV2Price")
price[:5]

[<span class="homecardV2Price" data-rf-test-name="homecard-price">$325,000</span>,
 <span class="homecardV2Price" data-rf-test-name="homecard-price">$400,000</span>,
 <span class="homecardV2Price" data-rf-test-name="homecard-price">$274,900</span>,
 <span class="homecardV2Price" data-rf-test-name="homecard-price">$489,900</span>,
 <span class="homecardV2Price" data-rf-test-name="homecard-price">$589,900</span>]

In [22]:
# Numebr of Beds, Baths and Sq. Ft. of listed properties
stats = soup.find_all('div', class_="stats")
stats[:5]

[<div class="stats">4 Beds</div>,
 <div class="stats">2 Baths</div>,
 <div class="stats">2,018 Sq. Ft.</div>,
 <div class="stats">4 Beds</div>,
 <div class="stats">3 Baths</div>]

In [23]:
# Zipcode of listed properties
zipcode = soup.find_all('div', class_="link-and-anchor")
zipcode[:5]

[<div class="link-and-anchor">11509 Parade Dr, Frisco, TX 75036</div>,
 <div class="link-and-anchor">12759 Blue Ridge Dr, Frisco, TX 75033</div>,
 <div class="link-and-anchor">13003 Michelle Dr, Frisco, TX 75035</div>,
 <div class="link-and-anchor">1186 Shortgrass Ln, Frisco, TX 75033</div>,
 <div class="link-and-anchor">8387 Pitkin Rd, Frisco, TX 75036</div>]

### 2. Data Cleaning and Transformation to load into DB

In [24]:
# Convering Price value into numeric - removing "$" and ","
price_list = [float(p.text.replace("$","").replace(",","").replace("+","").strip()) for p in price]
price_list[:5]

[325000.0, 400000.0, 274900.0, 489900.0, 589900.0]

In [25]:
print("Number of records in price list", len(price_list))

Number of records in price list 41


In [26]:
# Convering Number of Beds, Number of Baths and Area Sq. Ft. values into numeric - removing text: "Beds", "Baths", "Sq. Ft."
beds_list = []
baths_list = []
sq_ft_list = []

for each in stats:
    val = each.text.strip()
    if 'Bed' in val:
        beds_list.append(float(val.replace("Beds","").replace("Bed","").strip()))
    elif 'Bath' in val:  
        baths_list.append(float(val.replace("Baths","").replace("Bath","").strip()))
    elif 'Sq. Ft.' in val:
        sq_ft_list.append(float(val.replace("Sq. Ft.","").replace(",","").strip()))

print("Beds List:", beds_list[:5])
print("Baths List:", baths_list[:5])
print("Sq. Ft. List:", sq_ft_list[:5])

Beds List: [4.0, 4.0, 3.0, 4.0, 4.0]
Baths List: [2.0, 3.0, 2.0, 3.5, 4.0]
Sq. Ft. List: [2018.0, 3196.0, 1660.0, 3576.0, 3886.0]


In [27]:
print("Number of records in beds list", len(beds_list))
print("Number of records in baths list", len(baths_list))
print("Number of records in sq. ft. list", len(sq_ft_list))

Number of records in beds list 41
Number of records in baths list 41
Number of records in sq. ft. list 41


In [28]:
#Extracting zipcode from the complete address
zipcode_list = []
zipcode_list =  [z.text[-5:].strip() for z in zipcode]
print("Zipcode List:", zipcode_list[:5])

Zipcode List: ['75036', '75033', '75035', '75033', '75036']


In [29]:
print("Number of records in zipcode list", len(zipcode_list))

Number of records in zipcode list 41


#### Creating a list of Dictionary with current listed real estate properties to load into MongoDB

In [30]:
sold_listing = []

for i in range(len(zipcode_list)):
    temp_dict = {"ZipCode":zipcode_list[i], "NumberOfBeds": beds_list[i], "NumberOfBaths": baths_list[i], 
           "SqFt": sq_ft_list[i], "Price":price_list[i]}
    sold_listing.append(temp_dict)
    
print("Records to insert in DB:")
print(sold_listing[:5])

Records to insert in DB:
[{'ZipCode': '75036', 'NumberOfBeds': 4.0, 'NumberOfBaths': 2.0, 'SqFt': 2018.0, 'Price': 325000.0}, {'ZipCode': '75033', 'NumberOfBeds': 4.0, 'NumberOfBaths': 3.0, 'SqFt': 3196.0, 'Price': 400000.0}, {'ZipCode': '75035', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.0, 'SqFt': 1660.0, 'Price': 274900.0}, {'ZipCode': '75033', 'NumberOfBeds': 4.0, 'NumberOfBaths': 3.5, 'SqFt': 3576.0, 'Price': 489900.0}, {'ZipCode': '75036', 'NumberOfBeds': 4.0, 'NumberOfBaths': 4.0, 'SqFt': 3886.0, 'Price': 589900.0}]


In [31]:
browser.quit()

### 3. Loading data into Mongo DB

In [32]:
# collection 'sold_listing'
sold_listing_collection = db.sold_listing_collection

In [33]:
# Loading Data into collection table
for i in sold_listing:
    db.sold_listing_collection.insert_one(i)

In [34]:
# checking loaded records
for record in db.current_listing_collection.find():
    print(record)

{'_id': ObjectId('6062623af14bffb3486fd800'), 'ZipCode': '75036', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.5, 'SqFt': 2997.0, 'Price': 564900.0}
{'_id': ObjectId('6062623af14bffb3486fd801'), 'ZipCode': '75034', 'NumberOfBeds': 3.0, 'NumberOfBaths': 3.5, 'SqFt': 4228.0, 'Price': 930000.0}
{'_id': ObjectId('6062623af14bffb3486fd802'), 'ZipCode': '75036', 'NumberOfBeds': 2.0, 'NumberOfBaths': 2.0, 'SqFt': 1548.0, 'Price': 305000.0}
{'_id': ObjectId('6062623af14bffb3486fd803'), 'ZipCode': '75033', 'NumberOfBeds': 5.0, 'NumberOfBaths': 3.5, 'SqFt': 3556.0, 'Price': 549900.0}
{'_id': ObjectId('6062623af14bffb3486fd804'), 'ZipCode': '75036', 'NumberOfBeds': 3.0, 'NumberOfBaths': 2.0, 'SqFt': 1970.0, 'Price': 423550.0}
{'_id': ObjectId('6062623af14bffb3486fd805'), 'ZipCode': '75035', 'NumberOfBeds': 4.0, 'NumberOfBaths': 3.0, 'SqFt': 3676.0, 'Price': 599900.0}
{'_id': ObjectId('6062623af14bffb3486fd806'), 'ZipCode': '75036', 'NumberOfBeds': 5.0, 'NumberOfBaths': 3.5, 'SqFt': 3936.0, 'Price': 52